In [31]:
import fastembed

model_handle = 'jinaai/jina-embeddings-v2-small-en'
text_embedding = fastembed.TextEmbedding(model_handle)
q_str = "I just discovered the course. Can I join now?"
q = text_embedding.embed(q_str)
q = list(q)[0]
min(q)  # q1 answer

-0.11726373885183883

In [4]:
import numpy as np

doc = text_embedding.embed("Can I still join the course after the start date?")
doc = list(doc)[0]
q.dot(doc)  # q2 answer

0.9008528895674547

In [5]:
documents_raw = [
    {
        "text": "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
        "section": "General course-related questions",
        "question": "Course - Can I still join the course after the start date?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.",
        "section": "General course-related questions",
        "question": "Course - Can I follow the course after it finishes?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.",
        "section": "General course-related questions",
        "question": "Course - What can I do before the course starts?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.",
        "section": "General course-related questions",
        "question": "How can we contribute to the course?",
        "course": "data-engineering-zoomcamp",
    },
]

documents = text_embedding.embed([doc["text"] for doc in documents_raw])
documents = np.array(list(documents))
print(documents.dot(q).argmax())  # documents.dot(q).argsort()[::-1] # q3 answer

1


In [6]:
[
    doc.update({"full_text": doc["question"] + " " + doc["text"]})
    for doc in documents_raw
]
documents = text_embedding.embed([doc["full_text"] for doc in documents_raw])
documents = np.array(list(documents))
print(documents.dot(q).argmax())  # documents.dot(q).argsort()[::-1] # q4 answer

0


In [7]:
print(documents.dot(q))

[0.85145432 0.84365942 0.8408287  0.7755158  0.80860078]


In [35]:
from fastembed import TextEmbedding

embedding_models = TextEmbedding.list_supported_models()
embedding_models.sort(key=lambda x: x["dim"])
EMBEDDING_DIMENSIONALITY = min_dim = embedding_models[0]["dim"]  # q5 answer
[
    (m["model"], m["size_in_GB"])
    for m in list(filter(lambda x: x["dim"] == min_dim, embedding_models))
]
model_handle = 'BAAI/bge-small-en'

In [25]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [28]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

# Define the collection name
collection_name = "ml-zoomcamp-rag"
client.delete_collection(collection_name)

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [29]:
points = [
    models.PointStruct(
        id=id_,
        vector=models.Document(
            text=doc["question"] + " " + doc["text"], model=model_handle
        ),
        # payload={
        #     "text": doc["text"],
        #     "section": doc["section"],
        #     "course": course["course"],
        # },  # save all needed metadata fields
    )
    for id_, doc in enumerate(documents)
]

client.upsert(collection_name=collection_name, points=points)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [36]:
client.query_points(collection_name, models.Document(text=q_str, model=model_handle))

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.8703172, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=6, version=0, score=0.86918855, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=13, version=0, score=0.86833113, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=293, version=0, score=0.8576106, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=17, version=0, score=0.85715395, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=231, version=0, score=0.85681665, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=262, version=0, score=0.85447824, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=9, version=0, score=0.84559387, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=5, version=0, score=0.8414748, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoi